<a href="https://colab.research.google.com/github/officialamit558/PyTorch/blob/main/PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to PyTorch


### Quick Start

In [2]:
# Import the required library

import torch # First download the torch from pip
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [3]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

100%|██████████| 26.4M/26.4M [00:02<00:00, 9.23MB/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29.5k/29.5k [00:00<00:00, 198kB/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4.42M/4.42M [00:01<00:00, 3.74MB/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5.15k/5.15k [00:00<00:00, 5.85MB/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [4]:
# Use the data loader which are download using the fashionmnist datasets

batch_size = 64

# Create data loader

train_dataloader = DataLoader(training_data , batch_size=batch_size)
test_dataloader = DataLoader(test_data , batch_size=batch_size)

for X , y in test_dataloader:
  print(f"Shape of X[N , C , H , W]: {X.shape} {X.dtype}")
  print(f"shape of y: {y.shape} {y.dtype}")
  print(len(X))
  print(len(y))
  break

Shape of X[N , C , H , W]: torch.Size([64, 1, 28, 28]) torch.float32
shape of y: torch.Size([64]) torch.int64
64
64


In [5]:
# Creating the model by usinf the torchvision
# We used the "Accelerator" for speed the training of the neural network.

if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")

print(f"Using {device} device")

# Define model

class NeuralNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28 , 512),
        nn.ReLU(),
        nn.Linear(512 , 512),
        nn.ReLU(),
        nn.Linear(512 , 10)

    )

  def forward(self , x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits

model = NeuralNetwork().to(device)
print(model)


Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [6]:
# So after creation of the model . try to optimize there parameter.
# for train model , we need the loss function and optimizer.
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr = 0.001)

# Make the function to train the model on the respective dataset.
def train(dataloader , model , loss_fn , optimizer):
  size = len(dataloader.dataset)
  model.train()
  for batch , (X,y) in enumerate(dataloader):
    X , y = X.to(device) , y.to(device)

    # Compute the prediction error
    pred = model(X)
    loss = loss_fn(pred , y)

    # Backpropagation
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if batch % 100 == 0:
      loss , current = loss.item() , (batch + 1) * len(X)
      print(f"loss:{loss:>7f} [{current:>5d}/{size:>5d}]")

In [7]:
# we also check the model performace on the test datatest.
def test(dataloader , model , loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  model.eval()
  test_loss , correct = 0 , 0
  with torch.no_grad():
    for X , y in dataloader:
      X , y = X.to(device) , y.to(device)
      pred = model(X)
      test_loss += loss_fn(pred , y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()
  test_loss /= num_batches
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [8]:
# Now our task of defining the train and test function for the training and testing of the given data

# first of all we define the number of epochs on our batch size data train
epochs = 5
for t in range(epochs):
  print(f"Epoch {t+1}\n ------------------------")
  train(train_dataloader , model , loss_fn , optimizer)
  test(test_dataloader , model , loss_fn)
print("Done")

Epoch 1
 ------------------------
loss:2.297401 [   64/60000]
loss:2.288294 [ 6464/60000]
loss:2.262791 [12864/60000]
loss:2.257319 [19264/60000]
loss:2.245998 [25664/60000]
loss:2.208615 [32064/60000]
loss:2.220539 [38464/60000]
loss:2.181887 [44864/60000]
loss:2.172261 [51264/60000]
loss:2.136672 [57664/60000]
Test Error: 
 Accuracy: 41.7%, Avg loss: 2.136441 

Epoch 2
 ------------------------
loss:2.141582 [   64/60000]
loss:2.139415 [ 6464/60000]
loss:2.070297 [12864/60000]
loss:2.092332 [19264/60000]
loss:2.040942 [25664/60000]
loss:1.974066 [32064/60000]
loss:2.001434 [38464/60000]
loss:1.915469 [44864/60000]
loss:1.918906 [51264/60000]
loss:1.841565 [57664/60000]
Test Error: 
 Accuracy: 59.8%, Avg loss: 1.844959 

Epoch 3
 ------------------------
loss:1.871852 [   64/60000]
loss:1.851310 [ 6464/60000]
loss:1.721247 [12864/60000]
loss:1.773221 [19264/60000]
loss:1.661434 [25664/60000]
loss:1.615276 [32064/60000]
loss:1.632323 [38464/60000]
loss:1.532637 [44864/60000]
loss:1.565

In [9]:
# Now time to save the model for use case in the real life.
# A common way of save a model is to serialize the internal state dictionary which is the model parameters.

torch.save(model.state_dict(), "model.pth")
print("Save pytorch model state to model.pth")


Save pytorch model state to model.pth


In [20]:
# Check our model for make predictions
# Load the trained model
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load('/content/model.pth' , weights_only = True))

classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x , y = test_data[2][0] , test_data[2][1]
x= x.unsqueeze(0)
with torch.no_grad():
  x = x.to(device)
  print(x)
  print(len(x))
  pred = model(x)
  predicted , actual = classes[pred[0].argmax(0)] , classes[y]
  print(f"Predicted: {predicted} , Original: {actual}")

tensor([[[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0039, 0.0000, 0.2627, 0.6941, 0.5059, 0.6000, 0.4588, 0.5059,
           0.5725, 0.5529, 0.6863, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0078, 0.0000, 0.7686, 1.0000, 1.0000, 1.0000, 0.9451, 0.9843,
           1.0000, 0.9608, 1.0000, 0.2980, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.9529, 0.9294, 0.8510, 0.8941, 0.9059, 0.8706,
           0.8549, 0.8588, 1.0000, 0.4549, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 1.0000, 0.9216, 0.9059, 0.9137, 0.8863, 0.8824,
           0.8980, 0.8706, 1.0000, 0.5686, 

In [30]:
for i in range(len(test_data)):
  for j in range(len(test_data)):
    if i == 0 and j == 1:
      print(test_data[i][j])
      break

9


### *Tensors*

In [3]:
# Try to understand the tensor with some examples
import numpy as np
import torch

In [27]:
# Initilizing a tensor
# Directly from data
data = [[1 , 2],[3 , 4]] # This 2D metrics
x_data = torch.tensor(data)
print(x_data)
type(x_data)
# type(data)

tensor([[1, 2],
        [3, 4]])


torch.Tensor

In [28]:
# Tensor can created using ndarray

np_array = np.array(data)
x_np = torch.from_numpy(np_array)
print(x_np)

tensor([[1, 2],
        [3, 4]])


In [29]:
# From another tensor
x_ones = torch.ones_like(x_data)
x_rand = torch.rand_like(x_data , dtype = torch.float)
print(f"ones tensor: {x_data}")
print(f"random tensor: {x_rand}")

ones tensor: tensor([[1, 2],
        [3, 4]])
random tensor: tensor([[0.1819, 0.0653],
        [0.1840, 0.6039]])


In [31]:
# With random or constant values
shape = (4 , 5,)

rand_tensor = torch.rand(shape)
ones_tensor = torch.ones(shape)
zeros_tensor = torch.zeros(shape)

print(f"Random Tensor: {rand_tensor}")
print(f"Ones Tensor: {ones_tensor}")
print(f"Zeros Tensor: {zeros_tensor}")

Random Tensor: tensor([[0.1199, 0.2277, 0.4503, 0.5660, 0.7424],
        [0.2138, 0.5192, 0.6906, 0.5922, 0.6765],
        [0.6123, 0.2124, 0.5766, 0.6059, 0.8142],
        [0.8456, 0.0089, 0.0835, 0.9716, 0.7915]])
Ones Tensor: tensor([[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]])
Zeros Tensor: tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])


In [33]:
if torch.cuda.is_available():
  device = torch.device('cuda')
else:
  device = torch.device('cpu')

In [4]:
# Attributes of a tensor
tensor = torch.rand(3,4)
print(f"Shape of tensor: {tensor.shape}")
print(f"Datatype of tensor: {tensor.dtype}")
print(f"Device tensor is stored on: {tensor.device}")

Shape of tensor: torch.Size([3, 4])
Datatype of tensor: torch.float32
Device tensor is stored on: cpu


In [5]:
# Operations on the tensors
tensor = torch.ones(4,4)
print(f"First row: {tensor[0]}")
print(f"First columns: {tensor[:,0]}")
print(f"Last column: {tensor[... , -1]}")

print(tensor)

First row: tensor([1., 1., 1., 1.])
First columns: tensor([1., 1., 1., 1.])
Last column: tensor([1., 1., 1., 1.])
tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])


In [6]:
# Joining tensors
tensor[: , 1] = 0
print(tensor)

tensor([[1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.]])


In [7]:
# we can join the more than one tensor by using the torch.cat or torch.stack method

t1 = torch.cat([tensor , tensor , tensor] , dim = 1)
print(t1)

tensor([[1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.],
        [1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.],
        [1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.],
        [1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.]])


In [8]:
# Arithmetic operatins on tensors
# tensor.T it return the transpose of the two matrix
y1 = tensor @ tensor.T
y2 = tensor.matmul(tensor.T)

y3 = torch.rand_like(y1)
torch.matmul(tensor , tensor.T , out=y3)

# This compute the element wise product . z1 , z2 , z3 will have the same value
z1 = tensor * tensor
z2 = tensor.mul(tensor)

z3 = torch.rand_like(tensor)
torch.mul(tensor , tensor , out = z3)

print(f'Transpose operation is : {y1}')
print(f'Element wise multiplication : {z1}')

Transpose operation is : tensor([[3., 3., 3., 3.],
        [3., 3., 3., 3.],
        [3., 3., 3., 3.],
        [3., 3., 3., 3.]])
Element wise multiplication : tensor([[1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.]])
